# Multi-Factor Stock Screener (Value + Momentum + Quality)

This notebook combines:
- **Value**: Low P/E relative to market cap (original strategy)
- **Momentum**: Positive 6-month price performance
- **Quality**: Positive historical earnings growth, avoiding value traps

In [1]:
from finvizfinance.screener.overview import Overview
from finvizfinance.screener.valuation import Valuation
from finvizfinance.screener.performance import Performance

# Common filters
filters_dict = {
    'Exchange':'Any',
    'Country': 'USA',
    'Market Cap.': '+Large (over $10bln)',
    'Industry': 'Stocks only (ex-Funds)',
}

# Get valuation data
fvaluation = Valuation()
fvaluation.set_filter(filters_dict=filters_dict)
df_valuation = fvaluation.screener_view()

print(f"Valuation data: {df_valuation.shape}")

Valuation data: (713, 17)#########################-] 35/36 


In [2]:
# Get performance data for momentum
fperformance = Performance()
fperformance.set_filter(filters_dict=filters_dict)
df_performance = fperformance.screener_view()

print(f"Performance data: {df_performance.shape}")
print(f"Performance columns: {df_performance.columns.tolist()}")

Performance data: (713, 17)#######################-] 35/36 
Performance columns: ['Ticker', 'Perf Week', 'Perf Month', 'Perf Quart', 'Perf Half', 'Perf YTD', 'Perf Year', 'Perf 3Y', 'Perf 5Y', 'Perf 10Y', 'Volatility W', 'Volatility M', 'Avg Volume', 'Rel Volume', 'Price', 'Change', 'Volume']


In [4]:
# Merge valuation and performance data
df = df_valuation.merge(
    df_performance[['Ticker', 'Perf Half', 'Perf Year', 'Perf Quart', 'Volatility M']], 
    on='Ticker', 
    how='left'
)

print(f"Merged data: {df.shape}")
df.head()

Merged data: (713, 21)


,Ticker,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,EPS This Y,...,EPS Past 5Y,EPS Next 5Y,Sales Past 5Y,Price,Change,Volume,Perf Half,Perf Year,Perf Quart,Volatility M
0,A,3.962000e+10,30.59,21.38,2.34,5.70,5.87,22.15,34.40,6.16%,...,14.68%,9.13%,5.41%,139.77,0.0290,1987701.0,0.2029,-0.0515,-0.0226,0.0220
1,AA,1.654000e+10,14.51,14.57,0.27,1.26,2.61,10.22,31.87,164.16%,...,-,54.67%,3.03%,63.87,0.0282,6486936.0,1.0003,0.6157,0.6394,0.0390
2,AAL,1.000000e+10,18.21,7.68,0.75,0.18,NaN,1.31,11.34,-71.57%,...,-20.93%,10.21%,3.44%,15.15,0.0243,51995484.0,0.2120,-0.1708,0.2449,0.0341
3,AAPL,3.639940e+12,33.20,27.17,2.61,8.75,49.62,66.55,36.85,10.34%,...,17.91%,10.40%,8.71%,247.65,0.0039,54139306.0,0.1551,0.0768,-0.0556,0.0169
4,ABBV,3.820200e+11,163.10,15.18,0.93,6.41,NaN,67.36,19.41,2.22%,...,-14.67%,16.26%,11.11%,216.15,0.0099,10705074.0,0.1552,0.2599,-0.0686,0.0232


In [5]:
# Sort by market cap
sorted_stocks = df.sort_values('Market Cap', ascending=False)
print(f"Total stocks: {sorted_stocks.shape[0]}")
sorted_stocks.head()

Total stocks: 713


,Ticker,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,EPS This Y,...,EPS Past 5Y,EPS Next 5Y,Sales Past 5Y,Price,Change,Volume,Perf Half,Perf Year,Perf Quart,Volatility M
460,NVDA,4.454680e+12,45.41,23.70,0.48,23.80,37.47,73.50,57.61,56.57%,...,91.83%,49.28%,64.24%,183.32,0.0295,198837362.0,0.0975,0.3312,0.0037,0.0239
299,GOOGL,3.963440e+12,32.39,29.15,1.67,10.28,10.25,40.24,53.89,32.24%,...,26.76%,17.48%,16.73%,328.38,0.0198,35269049.0,0.7162,0.6754,0.2800,0.0212
298,GOOG,3.961480e+12,32.39,29.15,1.67,10.28,10.25,40.22,53.86,32.24%,...,26.76%,17.48%,16.73%,328.38,0.0193,22689044.0,0.7093,0.6623,0.2776,0.0211
3,AAPL,3.639940e+12,33.20,27.17,2.61,8.75,49.62,66.55,36.85,10.34%,...,17.91%,10.40%,8.71%,247.65,0.0039,54139306.0,0.1551,0.0768,-0.0556,0.0169
433,MSFT,3.300790e+12,31.60,23.65,1.33,11.23,9.09,32.36,42.31,18.03%,...,18.80%,17.77%,14.52%,444.11,-0.0229,37767504.0,-0.1210,0.0351,-0.1406,0.0156


In [6]:
# FILTER 1: Positive P/E (profitable companies only)
positive_pe = sorted_stocks[sorted_stocks['P/E'] > 0]
print(f"After positive P/E filter: {positive_pe.shape[0]} stocks")

After positive P/E filter: 629 stocks


In [7]:
# FILTER 2: Value filter - P/E less than half of market cap in billions
value_stocks = positive_pe[positive_pe['P/E'] < (0.5 * positive_pe['Market Cap'] / 1_000_000_000)]
print(f"After value filter: {value_stocks.shape[0]} stocks")

After value filter: 214 stocks


In [9]:
# Helper function to convert percentage strings to floats
def pct_to_float(series):
    """Convert percentage strings like '15.5%' to float 0.155"""
    def convert(val):
        if pd.isna(val) or val == '-' or val == '':
            return float('nan')
        if isinstance(val, str):
            return float(val.replace('%', '')) / 100
        return val
    return series.apply(convert)

import pandas as pd

# Convert percentage columns to numeric
value_stocks = value_stocks.copy()
value_stocks['EPS_Growth_5Y'] = pct_to_float(value_stocks['EPS Past 5Y'])
value_stocks['Momentum_6M'] = pct_to_float(value_stocks['Perf Half'])
value_stocks['Momentum_1Y'] = pct_to_float(value_stocks['Perf Year'])

print("Converted percentage columns to numeric")
value_stocks[['Ticker', 'EPS_Growth_5Y', 'Momentum_6M', 'Momentum_1Y']].head(10)

Converted percentage columns to numeric


,Ticker,EPS_Growth_5Y,Momentum_6M,Momentum_1Y
460,NVDA,0.9183,0.0975,0.3312
299,GOOGL,0.2676,0.7162,0.6754
298,GOOG,0.2676,0.7093,0.6623
3,AAPL,0.1791,0.1551,0.0768
433,MSFT,0.1880,-0.1210,0.0351
40,AMZN,0.3689,0.0169,0.0238
58,AVGO,0.4976,0.1802,0.3848
413,META,0.2999,-0.1303,0.0003
626,TSLA,NaN,0.2991,0.0116
91,BRK-B,0.0443,0.0119,0.0339


In [10]:
# FILTER 3: Quality filter - Positive earnings growth over past 5 years
# This helps avoid value traps (cheap stocks that are cheap for a reason)
quality_stocks = value_stocks[value_stocks['EPS_Growth_5Y'] > 0]
print(f"After quality filter (positive EPS growth): {quality_stocks.shape[0]} stocks")

After quality filter (positive EPS growth): 166 stocks


In [11]:
# FILTER 4: Momentum filter - Positive 6-month price performance
# Research shows combining value with momentum significantly improves returns
momentum_stocks = quality_stocks[quality_stocks['Momentum_6M'] > 0]
print(f"After momentum filter (positive 6M performance): {momentum_stocks.shape[0]} stocks")

After momentum filter (positive 6M performance): 112 stocks


In [14]:
# Create analysis dataframe with key columns
analysis_stocks = momentum_stocks[[
    'Ticker', 'Price', 'Market Cap', 'P/E', 
    'EPS Past 5Y', 'Perf Half', 'Perf Year'
]].copy()

# Calculate earnings in billions
analysis_stocks['Earnings (in B)'] = analysis_stocks['Market Cap'] / (1_000_000_000 * analysis_stocks['P/E'])

# Rename columns for clarity
analysis_stocks = analysis_stocks.rename(columns={
    'EPS Past 5Y': 'Earnings Growth (5Y)',
    'Perf Half': 'Momentum (6M)',
    'Perf Year': 'Momentum (1Y)'
})

# Reorder columns
analysis_stocks = analysis_stocks[[
    'Ticker', 'Price', 'Market Cap', 'P/E', 'Earnings (in B)',
    'Earnings Growth (5Y)', 'Momentum (6M)', 'Momentum (1Y)'
]]

print(f"Analysis stocks: {analysis_stocks.shape[0]}")
analysis_stocks.head(15)

Analysis stocks: 112


,Ticker,Price,Market Cap,P/E,Earnings (in B),Earnings Growth (5Y),Momentum (6M),Momentum (1Y)
460,NVDA,183.32,4.454680e+12,45.41,98.099097,91.83%,0.0975,0.3312
299,GOOGL,328.38,3.963440e+12,32.39,122.366162,26.76%,0.7162,0.6754
298,GOOG,328.38,3.961480e+12,32.39,122.305650,26.76%,0.7093,0.6623
3,AAPL,247.65,3.639940e+12,33.20,109.636747,17.91%,0.1551,0.0768
40,AMZN,231.31,2.472750e+12,32.68,75.665545,36.89%,0.0169,0.0238
58,AVGO,328.80,1.558930e+12,69.06,22.573559,49.76%,0.1802,0.3848
91,BRK-B,483.83,1.043480e+12,15.47,67.451842,4.43%,0.0119,0.0339
90,BRK-A,724079.50,1.041920e+12,15.44,67.481865,4.43%,0.0131,0.0303
386,LLY,1078.52,1.019620e+12,53.34,19.115486,18.77%,0.3891,0.4861
688,WMT,119.36,9.513200e+11,41.87,22.720802,6.82%,0.2453,0.2982


In [15]:
# Rank stocks by a composite score (optional enhancement)
# Higher momentum + higher earnings growth = better rank
ranked_stocks = analysis_stocks.copy()

# Convert to numeric for ranking
ranked_stocks['Growth_Numeric'] = pct_to_float(ranked_stocks['Earnings Growth (5Y)'])
ranked_stocks['Mom_6M_Numeric'] = pct_to_float(ranked_stocks['Momentum (6M)'])

# Create percentile ranks (higher is better)
ranked_stocks['Value_Rank'] = (1 / ranked_stocks['P/E']).rank(pct=True)  # Lower P/E = higher rank
ranked_stocks['Growth_Rank'] = ranked_stocks['Growth_Numeric'].rank(pct=True)
ranked_stocks['Momentum_Rank'] = ranked_stocks['Mom_6M_Numeric'].rank(pct=True)

# Composite score: equal weight to all three factors
ranked_stocks['Composite_Score'] = (
    ranked_stocks['Value_Rank'] + 
    ranked_stocks['Growth_Rank'] + 
    ranked_stocks['Momentum_Rank']
) / 3

# Sort by composite score
ranked_stocks = ranked_stocks.sort_values('Composite_Score', ascending=False)

# Keep only display columns
final_stocks = ranked_stocks[[
    'Ticker', 'Price', 'Market Cap', 'P/E', 'Earnings (in B)',
    'Earnings Growth (5Y)', 'Momentum (6M)', 'Momentum (1Y)', 'Composite_Score'
]].copy()

print(f"Ranked stocks: {final_stocks.shape[0]}")
final_stocks.head(25)

Ranked stocks: 112


,Ticker,Price,Market Cap,P/E,Earnings (in B),Earnings Growth (5Y),Momentum (6M),Momentum (1Y),Composite_Score
248,F,13.77,5.487000e+10,11.81,4.646063,162.64%,0.2306,0.3744,0.872024
82,BKR,53.59,5.288000e+10,18.45,2.866125,66.90%,0.3391,0.1515,0.797619
254,FDX,306.95,7.217000e+10,16.93,4.262847,27.97%,0.3248,0.1158,0.776786
270,FOX,65.47,3.049000e+10,14.73,2.069925,24.80%,0.2745,0.4339,0.773810
425,MPLX,55.62,5.657000e+10,11.79,4.798134,32.50%,0.1025,0.0874,0.764881
249,FANG,153.00,4.384000e+10,10.63,4.124177,60.36%,0.0788,-0.1508,0.761905
198,DOV,208.55,2.860000e+10,12.82,2.230889,33.35%,0.1003,0.0690,0.744048
499,PHM,129.97,2.533000e+10,10.01,2.530470,32.08%,0.0726,0.1106,0.736607
299,GOOGL,328.38,3.963440e+12,32.39,122.366162,26.76%,0.7162,0.6754,0.696429
298,GOOG,328.38,3.961480e+12,32.39,122.305650,26.76%,0.7093,0.6623,0.693452


In [16]:
# Load S&P 500 constituents for company names and sectors
sheet = 'constituents'
file_name = 's&p500-constituents.xlsx'
constituents = pd.read_excel(file_name, sheet_name=sheet)
constituents.head()

,Ticker,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [17]:
# Join with constituents to get company names and sectors
joined_stocks = pd.merge(final_stocks, constituents, on='Ticker', how='left').copy()

# Reorder columns - put Name first
name_col = joined_stocks.pop('Name')
joined_stocks.insert(0, 'Name', name_col)

print(f"Final joined stocks: {joined_stocks.shape[0]}")
joined_stocks.head(25)

Final joined stocks: 112


,Name,Ticker,Price,Market Cap,P/E,Earnings (in B),Earnings Growth (5Y),Momentum (6M),Momentum (1Y),Composite_Score,Sector
0,Ford,F,13.77,5.487000e+10,11.81,4.646063,162.64%,0.2306,0.3744,0.872024,Consumer Discretionary
1,Baker Hughes,BKR,53.59,5.288000e+10,18.45,2.866125,66.90%,0.3391,0.1515,0.797619,Energy
2,FedEx,FDX,306.95,7.217000e+10,16.93,4.262847,27.97%,0.3248,0.1158,0.776786,Industrials
3,Fox Corporation (Class B),FOX,65.47,3.049000e+10,14.73,2.069925,24.80%,0.2745,0.4339,0.773810,Communication Services
4,NaN,MPLX,55.62,5.657000e+10,11.79,4.798134,32.50%,0.1025,0.0874,0.764881,NaN
5,Diamondback Energy,FANG,153.00,4.384000e+10,10.63,4.124177,60.36%,0.0788,-0.1508,0.761905,Energy
6,Dover Corporation,DOV,208.55,2.860000e+10,12.82,2.230889,33.35%,0.1003,0.0690,0.744048,Industrials
7,PulteGroup,PHM,129.97,2.533000e+10,10.01,2.530470,32.08%,0.0726,0.1106,0.736607,Consumer Discretionary
8,Alphabet (Class A),GOOGL,328.38,3.963440e+12,32.39,122.366162,26.76%,0.7162,0.6754,0.696429,Communication Services
9,Alphabet (Class C),GOOG,328.38,3.961480e+12,32.39,122.305650,26.76%,0.7093,0.6623,0.693452,Communication Services


In [18]:
# Get top 25 stocks for the strategy
top_25 = joined_stocks.head(25).copy()
print("=" * 60)
print("TOP 25 MULTI-FACTOR STOCKS (Value + Momentum + Quality)")
print("=" * 60)
top_25[['Name', 'Ticker', 'Price', 'P/E', 'Earnings Growth (5Y)', 'Momentum (6M)', 'Composite_Score']]

TOP 25 MULTI-FACTOR STOCKS (Value + Momentum + Quality)


,Name,Ticker,Price,P/E,Earnings Growth (5Y),Momentum (6M),Composite_Score
0,Ford,F,13.77,11.81,162.64%,0.2306,0.872024
1,Baker Hughes,BKR,53.59,18.45,66.90%,0.3391,0.797619
2,FedEx,FDX,306.95,16.93,27.97%,0.3248,0.776786
3,Fox Corporation (Class B),FOX,65.47,14.73,24.80%,0.2745,0.773810
4,NaN,MPLX,55.62,11.79,32.50%,0.1025,0.764881
5,Diamondback Energy,FANG,153.00,10.63,60.36%,0.0788,0.761905
6,Dover Corporation,DOV,208.55,12.82,33.35%,0.1003,0.744048
7,PulteGroup,PHM,129.97,10.01,32.08%,0.0726,0.736607
8,Alphabet (Class A),GOOGL,328.38,32.39,26.76%,0.7162,0.696429
9,Alphabet (Class C),GOOG,328.38,32.39,26.76%,0.7093,0.693452


In [19]:
# Summary statistics
print("\n" + "=" * 60)
print("PORTFOLIO SUMMARY")
print("=" * 60)
print(f"Number of stocks: {len(top_25)}")
print(f"Average P/E: {top_25['P/E'].mean():.2f}")
print(f"Median P/E: {top_25['P/E'].median():.2f}")
print(f"\nSector breakdown:")
print(top_25['Sector'].value_counts())


PORTFOLIO SUMMARY
Number of stocks: 25
Average P/E: 21.72
Median P/E: 17.94

Sector breakdown:
Sector
Energy                    4
Health Care               4
Financials                4
Consumer Discretionary    3
Communication Services    3
Information Technology    3
Industrials               2
Name: count, dtype: int64


In [20]:
from datetime import datetime
import openpyxl
import os

def save_excel_sheet(df, filepath, sheetname, index=False):
    """Save dataframe to Excel, creating file or adding sheet as needed."""
    if not os.path.exists(filepath):
        df.to_excel(filepath, sheet_name=sheetname, index=index)
    else:
        with pd.ExcelWriter(filepath, engine='openpyxl', if_sheet_exists='replace', mode='a') as writer:
            df.to_excel(writer, sheet_name=sheetname, index=index)

# Save all qualifying stocks
sheetname = datetime.now().strftime("%m-%d-%Y--%H-%M")
save_excel_sheet(joined_stocks, 'multifactor-stock-data.xlsx', sheetname)

# Also save top 25 to a separate sheet
save_excel_sheet(top_25, 'multifactor-stock-data.xlsx', f'Top25-{sheetname}')

print(f"Saved {len(joined_stocks)} stocks to 'multifactor-stock-data.xlsx'")
print(f"Sheet: {sheetname}")
print(f"Top 25 sheet: Top25-{sheetname}")

Saved 112 stocks to 'multifactor-stock-data.xlsx'
Sheet: 01-22-2026--01-54
Top 25 sheet: Top25-01-22-2026--01-54
